# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**<br>

- children — количество детей в семье<br>
- days_employed — общий трудовой стаж в днях<br>
- dob_years — возраст клиента в годах<br>
- education — уровень образования клиента<br>
- education_id — идентификатор уровня образования<br>
- family_status — семейное положение<br>
- family_status_id — идентификатор семейного положения<br>
- gender — пол клиента<br>
- income_type — тип занятости<br>
- debt — имел ли задолженность по возврату кредитов<br>
- total_income — ежемесячный доход<br>
- purpose — цель получения кредита<br>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [12]:
import pandas as pd
data = pd.read_csv('/Users/batorashaev/Desktop/data.csv')
#Изучим общую информацию о файле
data.info() 
display(data.head(10))
#print(data.tail(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Обнаружены пропущенные значения в стоблцах "Общий трудовой стаж в днях" и "Ежемесячный доход"

## Шаг 2. Предобработка данных

### Обработка пропусков

In [13]:
data['education'] = data['education'].str.lower()

#Высчитаем медианы по каждому типу занятости клиентов в зависимости от уровня образования:
#Для начала приведем столбец "Образование" к нижнему регистру:
data['education'] = data['education'].str.lower()
#Высчитаем медианы:
display(data.groupby(['income_type','education'])[['total_income']].agg(['median']))
#Заменим пустые значения на медианные:
data['total_income'] = data.groupby(['income_type','education'])['total_income'].apply(lambda x: x.fillna(x.median()))
#Заменим пропуски в столбце "Трудовой стаж" на "0", т.к. его значение не влияет на выдачу кредита:
data['days_employed'] = data['days_employed'].fillna(0) 
#Проверим, есть ли пропуски в столбцах:
print(data.isna().sum())

#display(data.head(20))
#data.info()

total_income
                                            median
income_type     education                         
безработный     высшее               202722.511368
                среднее               59956.991984
в декрете       среднее               53829.130729
госслужащий     высшее               172511.107016
                начальное            148339.290825
                неоконченное высшее  160592.345303
                среднее              136652.970357
                ученая степень       111392.231107
компаньон       высшее               201785.400018
                начальное            136798.905143
                неоконченное высшее  179867.152890
                среднее              159070.690289
пенсионер       высшее               144240.768611
                начальное            102598.653164
                неоконченное высшее  120136.896353
                среднее              114842.854099
                ученая степень       177088.845999
предприниматель высшее               499163.144947
сотрудник       высшее               165640.744634
                начальное            125994.910603
                неоконченное высшее  151308.937846
                среднее              136555.108821
                ученая степень       198570.757322
студент         высшее                98201.625314

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


In [14]:
data.groupby('income_type').agg({'total_income':'median'})

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,148978.800029
компаньон,170155.716801
пенсионер,114842.854099
предприниматель,499163.144947
сотрудник,138068.631966
студент,98201.625314


In [15]:
data.groupby('income_type').agg({'total_income':'median'}).loc['безработный'][0]

131339.7516762103

**Вывод**

Количетсво пропущенных значений по типу занятости:

Сотрудник: 1105 из 11119

Пенсионер: 413 из 3856

Компаньон: 508 из 5085

Госслужащий: 147 из 1459

Безработный: 0 из 2

Предприниматель: 1 из 2

Студент: 0 из 1

В декрете: 0 из 1

Исходя из полученных данных можно сделать вывод:

1)Одинковое количество пропусков в обеих столбцах (Общий трудовой стаж и Ежемесячный доход) - либо это "студенты", которые слишком молоды для работы, либо "Безработные" - безработные на самом деле, или официально нетрудоустроенные или имеющие иные способы заработка, либо это люди, только устроившиеся на работу.
2)пропущенные значения не связаны непосредственно с типом занятости, как например с типом "Безработный" или "Студент"(у которых нет ни стажа, ни дохода), а были либо не указаны, либо утеряны(т.к. пропущенные значения имеются у таких категорий, как "Сотрудник", "Госслужащий" и т.д.). 
3)Заменять пропущенные значения на медианные либо средние некорректно, так как это может повлиять на результат одобрения кредита, и в следствие этого, возомжность его невозврата в дальнейшем.
4)Решено заменить проущенные значения на "0".

UPD:

1) Высчитал медианные значения в столбце "Total Income" для каждого типа занятости клиентов (сотрудник, пенсионер и т.д.)

2) Заменил пустые значения на медианные

UPD2:

1) Высчитал медианные значения в столбце "Total Income" для каждого типа занятости клиентов в зависимости от образования

2) Заменил пустые значения на медианные

### Замена типа данных

In [16]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#data.info()
#print(data.head(10))
#Обнаружены отрицательные значения в столбце "Общий трудовой стаж в днях", а положительные значения являются некорректными 
data_days_plus = data[data['days_employed'] > 0]
data_days_minus = data[data['days_employed'] < 0]
#print(data_days_plus)
#print(data_days_minus)
#Данные с отрицательными значениями выглядят правдоподобнее, выясним количество отрицательных и положительных данных:

print('Отрицательных значений:', data_days_minus['days_employed'].count())
print('Максимальное отрицательное значение:', data_days_minus['days_employed'].max(), 'в годах:', (data_days_minus['days_employed'].max() / 365))
print('Минимальное отрицательное значение:',data_days_minus['days_employed'].min(), 'в годах:', (data_days_minus['days_employed'].min() / 365)) 
print('Положительных значений:', data_days_plus['days_employed'].count())
print('Максимальное положительное значение:',data_days_plus['days_employed'].max(), ', ' 'в годах:', (data_days_plus['days_employed'].max() / 365))
print('Максимальное положительное значение:',data_days_plus['days_employed'].min(), ', ' 'в годах:', (data_days_plus['days_employed'].min() / 365)) 

#Заменим отрицательные значения на полоительные методом abs
data['days_employed'] = data['days_employed'].abs()
print(data.head(10))
data.info()

Отрицательных значений: 15906
Максимальное отрицательное значение: -24 в годах: -0.06575342465753424
Минимальное отрицательное значение: -18388 в годах: -50.37808219178082
Положительных значений: 3445
Максимальное положительное значение: 401755 , в годах: 1100.6986301369864
Максимальное положительное значение: 328728 , в годах: 900.6246575342466
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9 

**Вывод**

Заменил вещественный тип данных на целочисленный.
Обнаружены отрицательные значения в стольце "Общий трудовой стаж в днях", а положительные значения являются некорректными, т.к. в пересчете получаются не реалистичные цифры.
Данные с отрицательными значениями выглядят правдоподобнее, выяснил количество отрицательных и положительных данных - отриательных большинство.
Заменил отрицательные значения на положительные методом abs, теперь в таблице нет отрицательных значений.

### Обработка дубликатов

In [123]:
#print(data.duplicated().sum())
#print(data['education'].value_counts())
#print(data['family_status'].value_counts())
#print(data['gender'].value_counts())
#print(data['income_type'].value_counts())
#print(data['purpose'].value_counts())

#приведем данные к единому регистру
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
#проверим еще раз
#print(data.duplicated().sum())
#print(data['education'].value_counts())
#print(data['family_status'].value_counts())
#print(data['gender'].value_counts())
#print(data['income_type'].value_counts())
#print(data['purpose'].value_counts()) 

#print(data.duplicated().sum())
#print('Уникальные значения в Children:', data['children'].unique())
#print('Уникальные значения в Days Employed:', data['days_employed'].unique())
#print('Уникальные значения в Dob Years:', data['dob_years'].unique())
#print('Уникальные значения в Education:', data['education'].unique())
#print('Уникальные значения в Education ID:', data['education_id'].unique())
#print('Уникальные значения в Family Status:', data['family_status'].unique())
#print('Уникальные значения в Family Status ID:', data['family_status_id'].unique())
#print('Уникальные значения в Gender:', data['gender'].unique())
#print('Уникальные значения в Debt:', data['debt'].unique())
#print('Уникальные значения в Income Type:', data['income_type'].unique())
#print('Уникальные значения в Total Income:', data['total_income'].unique())
#print('Уникальные значения в Purpose:', data['purpose'].unique())
#print('Количество дубликатов:', data.duplicated().sum())
#data[data['children'] == -1]['children'].count()
#data[data['children'] == 20]['children'].count()
data['children'] = data['children'].replace(-1, 1)
#data[data['children'] == -1]['children'].count()
data['children'] = data['children'].replace(20, 2)
#data[data['children'] == 20]['children'].count()
#data['children'].value_counts()
#data[data['dob_years'] == 0]['dob_years'].count()
#data[data['gender'] == 'xna'].count()

**Вывод**

Был обнаружен 71 дубликат, удалять их не стал, т.к. дубликаты в столбцах могут являться совпадением для двух разных лиц

В столбце Children обнаружены значения "-1"(47 раз) и "20"(76 раз), -1 заменил на 1, 20 скорее всего опечатка(т.к. трудно представить 76 заявителей с количеством детей равным 20), заменил на "2"

В стоблце Dob Years есть значения возраста "0", встречаются 101 раз

В столбце Gender присутствует странное значение "xna"
Уникальные значения в столбцах Eduaction и Education ID стали равны, т.к. привел значения к единому нижнему регистру

В столбце Purpose много взаимозаменяемых значений

### Лемматизация

In [125]:
#Импорт библиотек
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
from pymystem3 import Mystem
m = Mystem()
#Печатем лемматизированный список уникальных значений, выделили 4 категории
purpose_unique = data['purpose'].unique()
lemmas = []
for string in purpose_unique:
    lemma = m.lemmatize(string)
    lemmas.append(lemma)
#print(lemmas, '\n')

data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)

print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      f   сотрудник     0        253875   
1   женат / замужем                 0      f   сотрудник     0        112080   
2   женат / замужем                 0      m

**Вывод**

Создан новый столбец purpose_lemma - с леммитизированными целями.

### Категоризация данных

In [126]:
#Пропишем функцию для категоризации целей кредита
def change_purpose(purpose):
    if ('жилье' in purpose) or ('недвижимость' in purpose):
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
#Создан новый столбец с получившимися данными по категориям
data['purpose_category'] = data['purpose_lemma'].apply(change_purpose)
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      f   сотрудник     0        253875   
1   женат / замужем                 0      f   сотрудник     0        112080   
2   женат / замужем                 0      m

**Вывод**

Цели получения кредита были поделены на 4 основные категории: Недвижимость, Образование, Автомобиль, Свадьба.
Создан новый столбец purpose_category - в нем каждой заявке присвоена определенная категория.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [127]:
#data.loc[(data['children']==0)&(data['debt']==0), 'debt'].count()
#data.loc[(data['children']>0)&(data['debt']==0), 'debt'].count()
#data.loc[(data['children']==0)&(data['debt']==1), 'debt'].count()
#data.loc[(data['children']>0)&(data['debt']==1), 'debt'].count()
data_w_children = data[data['children'] != 0]
data_no_children = data[data['children'] == 0]
#data_w_children.info()
#data_no_children.info()
data_w_children_debt = data_w_children['debt'].sum() / len(data_w_children)
data_no_children_debt = data_no_children['debt'].sum() / len(data_no_children)
print('Процент должников среди людей с детьми: {:.1%}'.format(data_w_children_debt))
print('Процент должников среди людей без детей: {:.1%}'.format(data_no_children_debt))

Процент должников среди людей с детьми: 9.2%
Процент должников среди людей без детей: 7.5%


**Вывод**

Процент должников среди людей с детьми: 9.2%

Процент должников среди людей без детей: 7.5%

Люди с детьми платят кредиты хуже, но в целом - на одном уровне (разница 1,7%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [128]:
#family_status_debt = data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count() * 100
#print(family_status_debt)
data_pivot_family_status = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
data_pivot_family_status['ratio в %'] = round((data_pivot_family_status[1] / (data_pivot_family_status[0] + data_pivot_family_status[1]) * 100), 1)

#data_pivot_family_status['ratio'] = data_pivot_family_status['ratio'].astype('int')
print(data_pivot_family_status)

debt                       0    1  ratio в %
family_status                               
в разводе               1110   85        7.1
вдовец / вдова           897   63        6.6
гражданский брак        3789  388        9.3
женат / замужем        11449  931        7.5
не женат / не замужем   2539  274        9.7


**Вывод**

Можно сделать вывод, что не состоящие в официальном браке люди более склонны к образованию задолженностей по кредитам.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [129]:
#print(data['total_income'].min())
#print(data['total_income'].max())
#print(data['total_income'].median())
#print(data['total_income'].mean())
def income_total(data):
    if data == 0:
        return '0'
    elif data <= 30000:
        return '0-30'
    elif data <= 50000:
        return '30-50'
    elif data <= 100000:
        return '50-100'
    elif data <= 150000:
        return '100-150'
    elif data <= 300000:
        return '150-300'
    elif data <= 500000:
        return '300-500'
    else:
        return '500+'
data['income_total_category'] = data['total_income'].apply(income_total)
#print(data)
data_pivot_income_total = data.pivot_table(index='income_total_category', columns='debt', values='total_income', aggfunc='count')
data_pivot_income_total['ratio в %'] = round((data_pivot_income_total[1] / (data_pivot_income_total[0] + data_pivot_income_total[1]) * 100), 1)
print(data_pivot_income_total)

debt                      0    1  ratio в %
income_total_category                      
0-30                     20    2        9.1
100-150                6413  616        8.8
150-300                7885  665        7.8
30-50                   329   21        6.0
300-500                1169   92        7.3
50-100                 3760  331        8.1
500+                    208   14        6.3


**Вывод**

Лучше всего показатели среди следующих категорий зарплат:
30-50 - 6.0 %;
500+ - 6.3 %;
150-300 - 7.8 %;
300-500 - 7.3 %;

Чаще всего имеют проблемы со своевременной оплатой следующий категории:
0-30 - 9.1 %;
100-150 - 8.8 %;
50-100 - 8.1 %

Можно сделать вывод, что лучше всего выплачивают кредиты люди с доходом 30-50 тыс.руб. и высокими (от 150 тыс.руб.) доходами.
Хуже всего дела обстоят у среднего класса (50-150 тыс.руб.). Категорию 0 и 0-30 не беру в расчет, ввиду неверных данных либо маленькой выборки.

- Как разные цели кредита влияют на его возврат в срок?

In [130]:
data_pivot_purpose = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose['ratio в %'] = round((data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1]) * 100), 1)
#data_pivot_purpose['ratio в %'] = data_pivot_purpose['ratio в %'].astype('int64')
print(data_pivot_purpose)

debt                  0    1  ratio в %
purpose_category                       
автомобиль         3912  403        9.3
недвижимость      10058  782        7.2
образование        3652  370        9.2
свадьба            2162  186        7.9


**Вывод**

Кредиты на меньшие(маленькие) суммы больше подвержены риску несвоевременной выплаты:

Автомобиль и образование - 9.3%

Кредиты на приобретение недвижимости и проведение свадьбы с наименьшим процентом невыплат - 7.2 и 7.9 соответственно%

Лучшая ситуация с оплатой кредитов на недвижимость(в большинстве случаев, скорее всего, ипотека) обуславилвается тем, что такие кредиты берут более осознанно, нежели чем автомобиль или образование, бОльшими штрафными санкциями и суммой платежа(и самого кредита) забыть про который сложнее, нежели маленькую сумму.

Меньший процент невыплат среди кредитов на свадьбу по сравнению с кредитами на автомобиль и образование скорее всего обуславливается быстрой окупаемостью(свадьбы).

## Шаг 4. Общий вывод

Процент должников среди людей с детьми: 9.2%
Процент должников среди людей без детей: 7.5%
Люди с детьми платят кредиты хуже, но в целом - на одном уровне (разница 1,7%)

Не состоящие в официальном браке люди более склонны к образованию задолженностей по кредитам - гражданский брак 9.3 %, а не состоящие в браке - 9.7 % должников, что почти на 2% выше, чем у женатых/замужних.

Лучше всего показатели по задолженности среди следующих категорий зарплат:

30-50 - 6.0 %;

500+ - 6.3 %;

150-300 - 7.8 %;

300-500 - 7.3 %;

Чаще всего имеют проблемы со своевременной оплатой следующий категории:

0-30 - 9.1 %;

100-150 - 8.8 %;

50-100 - 8.1 %

Можно сделать вывод, что лучше всего выплачивают кредиты люди с доходом 30-50 тыс.руб. и высокими (от 150 тыс.руб.) доходами.
Хуже всего дела обстоят у среднего класса (50-150 тыс.руб.). Категорию 0 и 0-30 не беру в расчет, ввиду неверных данных либо маленькой выборки.

Кредиты на меньшие(маленькие) суммы больше подвержены риску несвоевременной выплаты:

Автомобиль и образование - 9.3%

Кредиты на приобретение недвижимости и проведение свадьбы с наименьшим процентом невыплат - 7.2 и 7.9 соответственно%

Лучшая ситуация с оплатой кредитов на недвижимость(в большинстве случаев, скорее всего, ипотека) обуславилвается тем, что такие кредиты берут более осознанно, нежели чем автомобиль или образование, бОльшими штрафными санкциями и суммой платежа(и самого кредита) забыть про который сложнее, нежели маленькую сумму.

Меньший процент невыплат среди кредитов на свадьбу по сравнению с кредитами на автомобиль и образование скорее всего обуславливается быстрой окупаемостью(свадьбы).